In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel(r"..\data\final_lexicon_gpt.xlsx")
df = df.fillna('')
df.head(5)

,id,extracted_needs,extracted_users,needs_gpt,users_gpt,text,needs_label,users_label,lexicon_need
0,0,availab,man,"Customizability, Efficiency, Usability",no,nice and easy but folder create function be no...,"Simplicity, Customizability, Usability",no,"availability,"
1,1,mobil,man,no,no,no don t install this in your mobile not at al...,no,no,"mobility,"
2,2,effecti,,"Efficiency, Effectiveness, Usability",no,very speedy app easy to use app that be quick ...,"Efficiency, Effectiveness, Usability, Simplicity",no,"effectiveness,"
3,3,availab,,"Availability, Reliability",no,I absolutely love this app I have try other ap...,no,no,"availability,"
4,4,secur,cur,"securability, Privacy",General user,I be happy with the app but I have one problem...,securability,no,"securability,"


<h3>Results statistics</h3>

In [4]:
num_need_gpt = 0
need_gpt = []
num_user_gpt = 0
user_gpt = []
num_need_lex = 0
need_lex = []
num_user_lex = 0
user_lex = []
label_need_cont = 0
label_need = []
label_user_cont = 0
label_user = []
for index, row in df.iterrows():
    if row['needs_gpt'] != "no" and type(row['needs_gpt']) != float:
        temp = row['needs_gpt'].split(",")
        for i in temp:
            need_gpt.append(i.lower().lstrip())
        num_need_gpt += len(temp)
    if row['users_gpt'] != "no" and type(row['users_gpt']) != float:
        temp = row['users_gpt'].split(",")
        for i in temp:
            user_gpt.append(i.lower().lstrip())
        num_user_gpt += len(temp)
    if row['lexicon_need'] != "no" and row['lexicon_need'] != " " and type(row['lexicon_need']) != float:
        temp = row['lexicon_need'].split(",")
        for i in temp:
            if i != "":
                need_lex.append(i.lower().lstrip())
        num_need_lex += len(temp)
    if row['extracted_users'] != "no" and type(row['extracted_users']) != float:
        temp = row['extracted_users'].split(",")
        for i in temp:
            user_lex.append(i.lower().lstrip())
        num_user_lex += len(temp)
    if row['needs_label'] != "no" and row['needs_label'] != " " and type(row['needs_label']) != float:
        temp = row['needs_label'].split(",")
        for i in temp:
            if i != "":
                label_need.append(i.lower().lstrip())
        label_need_cont += len(temp)
    if row['users_label'] != "no" and row['users_label'] != " " and row['users_label'] != "" and type(row['users_label']) != float:
        temp = row['users_label'].split(",")
        for i in temp:
            label_user.append(i.lower().lstrip())
        label_user_cont += len(temp)

print(num_need_gpt,num_user_gpt,num_need_lex,num_user_lex, label_need_cont, label_user_cont)    
    

2817 383 2418 1850 2462 128


In [5]:
from collections import Counter

In [13]:
x = Counter(user_gpt)
x.most_common()

[('general user', 45),
 ('it professionals', 38),
 ('mobile users', 16),
 ('software developers', 14),
 ('android users', 14),
 ('storage administrators', 9),
 ('cloud storage user', 9),
 ('gerry degoo support', 7),
 ('file transfer users', 6),
 ('android user', 5),
 ('mobile device users', 5),
 ('mobile app user', 4),
 ('storage professionals', 4),
 ('crm users', 4),
 ('gerry degoo', 3),
 ('project managers', 3),
 ('music enthusiasts', 3),
 ('mobile app users', 3),
 ('backup users', 3),
 ('android and ios users', 2),
 ('iphone and android users', 2),
 ('android app users', 2),
 ('software architects', 2),
 ('music enthusiast', 2),
 ('free user', 2),
 ('mobile user', 2),
 ('cloud service user', 2),
 ('file management users', 2),
 ('cloud storage users', 2),
 ('pc users', 2),
 ('developers', 2),
 ('file sharing users', 2),
 ('general users', 2),
 ('enterprise users', 2),
 ('data center managers', 2),
 ('security-conscious users', 2),
 ('personal users', 2),
 ('linux users', 2),
 ('priva

<h3>Evaluation</h3>

In [4]:
import pandas as pd
import numpy as np
from bert_score import score
from rouge_score import rouge_scorer
import re

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

import nltk
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(reference, candidate):
    # Tokenize the strings into lists of words
    reference_tokens = nltk.word_tokenize(reference.lower())
    candidate_tokens = nltk.word_tokenize(candidate.lower())

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
    return bleu_score

if __name__ == "__main__":
    # Example strings
    reference_string = "advice on accommodation in Uig, Staffin, and Waternish, information on parking situation in Portree, information on restaurant availability in Portree"
    candidate_string = "parking information, restaurant informations"

    # Calculate BLEU score
    bleu_score = calculate_bleu_score(reference_string, candidate_string)
    print("BLEU Score:", bleu_score)

def bertscore(label, predicted):
    P, R, F1 = score(predicted, label,model_type="microsoft/deberta-xlarge-mnli",rescale_with_baseline=False,  lang="en", verbose=False)
    return P, R, F1

def rougescore(label, predicted):
    scores = scorer.score(label, predicted)
    P1 = scores["rouge1"][0]
    R1 = scores["rouge1"][1]
    F1 = scores["rouge1"][2]
    P2 = scores["rouge2"][0]
    R2 = scores["rouge2"][1]
    F2 = scores["rouge2"][2]
    P3 = scores["rougeL"][0]
    R3 = scores["rougeL"][1]
    F3 = scores["rougeL"][2]
    return P1, R1, F1, P1, R2, F2, P3, R3, F3


def evaluation(df, col_label, col_eval):
    n = len(df.index)
    prec_bert = 0
    rec_bert = 0
    f1_bert = 0
    p_rouge1 = 0
    r_rouge1 = 0
    f_rouge1 = 0
    p_rouge2 = 0
    r_rouge2 = 0
    f_rouge2 = 0
    p_rougeL = 0
    r_rougeL = 0
    f_rougeL = 0
    bleu_t = 0
    Prec, Rec, F_1 = bertscore(df[col_eval].tolist(), df[col_label].tolist())
    for index, row in df.iterrows():
        #Prec, Rec, F_1 = bertscore([row[col_label]], [row["text"]])
        P1, R1, F1, P2, R2, F2, P3, R3, F3 = rougescore(row[col_label], row[col_eval])
        bleu = calculate_bleu_score(row[col_label], row[col_eval])
        bleu_t += bleu
        #prec_bert += Prec
        #rec_bert += Rec
        #f1_bert += F_1
        p_rouge1 += P1
        r_rouge1 += R1
        f_rouge1 += F1
        p_rouge2 += P2
        r_rouge2 += R2
        f_rouge2 += F2
        p_rougeL += P3
        r_rougeL += R3
        f_rougeL += F3
    #prec_bert = round(prec_bert.item()/n, 2)
    #rec_bert = round(rec_bert.item()/n, 2)
    #f1_bert = round(f1_bert.item()/n, 2)
    bleu_t = round(bleu_t/n, 3)
    p_rouge1 = round(p_rouge1/n, 3)
    r_rouge1 = round(r_rouge1/n, 3)
    f_rouge1 = round(f_rouge1/n, 3)
    p_rouge2 = round(p_rouge2/n, 3)
    r_rouge2 = round(r_rouge2/n, 3)
    f_rouge2 = round(f_rouge2/n, 3)
    p_rougeL = round(p_rougeL/n, 3)
    r_rougeL = round(r_rougeL/n, 3)
    f_rougeL = round(f_rougeL/n, 3)
    return round(Prec.mean().item(), 3), round(Rec.mean().item(), 3), round(F_1.mean().item(), 3), f_rouge1, f_rouge2, f_rougeL, bleu_t




def prepare(col1):
    col = col1.str.replace(r',', lambda x: x.group().replace(',', ' '), regex=True)
    col = col.str.replace(r',', lambda x: x.group().replace('-', ' '), regex=True)
    col = col.str.lower()
    return col
    

BLEU Score: 0.02115696976989302


In [9]:
df["needs_gpt"] = prepare(df["needs_gpt"])
df["users_gpt"] = prepare(df["users_gpt"])
df["lexicon_need"] = prepare(df["lexicon_need"])
df["extracted_users"] = prepare(df["extracted_users"])
df["needs_label"] = prepare(df["needs_label"])
df["users_label"] = prepare(df["users_label"])


In [10]:
print("eval NEEDS GPT")
print(evaluation(df, "needs_label", "needs_gpt"))

eval NEEDS GPT


Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(0.737, 0.728, 0.73, 0.639, 0.354, 0.622, 0.671)


In [11]:
print("eval NEEDS LEX")
print(evaluation(df, "needs_label", "lexicon_need"))

eval NEEDS LEX


Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(0.412, 0.505, 0.447, 0.395, 0.033, 0.384, 0.286)


In [12]:
print("eval USERS LEX")
print(evaluation(df, "users_label", "extracted_users"))

eval USERS LEX


Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(0.043, 0.056, 0.046, 0.003, 0.0, 0.003, 0.004)


In [13]:
print("eval USERS GPT")
print(evaluation(df, "users_label", "users_gpt"))

eval USERS GPT


Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(0.255, 0.255, 0.255, 0.23, 0.034, 0.23, 0.23)
